In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time
import torch

class TestModule(nn.Module):

    def __init__(self, layer, nlayers, channels, int_channels=16, kernel_size=3):
        super(TestModule, self).__init__()
        
        kernel_size = ntuple(3, 3)
        
        layers = [layer(channels, int_channels, kernel_size)]
        for i in range(nlayers):
            layers.append(layer(int_channels, int_channels, kernel_size))
        layers.append(layer(int_channels, 1, kernel_size))
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        
        for layer in self.children():
            x = layer(x)
        return x

def time_model(model, num_iterations=10, bs=8, channels=32, shape=None):
    
    if shape is None:
        shape = (50, 50)
        
    model.cuda()

    tt = 0
    for i in range(num_iterations):
        inp = torch.rand(bs, channels, *shape).cuda()
        ti = time.time()
        res = model(inp)
        tf = time.time()
        tt += tf-ti
    tm = tt/num_iterations
    
    model.cpu()

    return tm

In [ ]:
#Conv2dCH(1, 2, (3,3), stride=1, bias=False)(img[None, None])   
inp = torch.rand(1, 1, 5, 5, 5)
res = Conv3dCH(1, 1, (3, 3, 3), stride=1, bias=False)(inp) 

In [ ]:
conv3d = nn.Conv3d(3, 32, (7, 7, 7))
conv3dch = Conv3dCH(3, 32, (7, 7, 7))
tm = TestModule(nn.Conv3d, nlayers=18, channels=1, int_channels=32)

res = time_model(tm, num_iterations=10, bs=2, channels=1, shape=(70, 70, 70))